In [1]:
import os
import zipfile
import json
from scipy import sparse
import numpy as np
import codecs
#reader = codecs.getreader("utf-8")

In [2]:
datadirectory="C:\\Users\\ahmad\\Desktop\\mmds\\lastfm_test.zip"
dataset = zipfile.ZipFile(datadirectory)

In [3]:
songId=0
t=0
g=50
beta=0.2
categories =["Hip-Hop"]
S =[]
id2songJsonMap = []
id2songIdMap = {}
fhat=[]
#limitingCounter = 1000

In [4]:

for i in dataset.namelist():
    if  i.endswith(".json"): #songId<limitingCounter and 
        f = json.loads(dataset.read(i).decode("utf-8"))
        if  f["track_id"] not in id2songIdMap.keys():
            id2songIdMap[f["track_id"]] = songId
            tempcategories = set(categories)#will keep removing tags from it
            newtags=[]
            for tag in f["tags"]:#looping over tags
                if int(tag[1]) >g:#filters out those <g
                    newtags.append(tag)#keeping >g
                    if tag[0] in tempcategories:#if it is in the tempcategories
                        tempcategories.remove(tag[0])#removing it
            f["tags"] = newtags
            id2songJsonMap.append(f)
            #songTags= [t[0] for t in f["tags"]]
            #if len(list(filter(lambda x: x in categories,songTags )) ) == len(categories):
            if len(tempcategories)==0:#meaning we encountered all of the tags
                S.append(songId)
                fhat.append(1)
            else:
                fhat.append(0)
            songId+=1 

In [5]:
fhat=(1/len(S))*np.array(fhat)

In [6]:
col=[]
row=[]
data=[]
for i in range( len(id2songJsonMap)):
    for s in id2songJsonMap[i]["similars"]:
        if s[0] in id2songIdMap.keys() and s[1]>=t:
            col.append(i)
            row.append(id2songIdMap[s[0]])
            data.append(1)
    
    
    

        
        
    

In [7]:
A = sparse.csr_matrix((data, (row, col)),shape=(104212, 104212))

In [8]:
r =np.random.rand(A.shape[0],1)
r=r/np.sum(r)

In [9]:
print(len(id2songJsonMap))
print (A.shape)
print(songId)
print (len(S))
print (songId)

104212
(104212, 104212)
104212
1214
104212


In [10]:
r = beta * A.multiply(r)+(1-beta)*fhat

In [11]:
# for i in range(songId):
#     #neighbors = A.getrow(i)
#     for j in  range(songId):
#         if A[i,j]==1:
#             neighbourRank = A.getcol(j).sum()
#             r[i] = beta*r[i]*(r[j]/neighbourRank)+((1-beta)*fhat[i])

{u'artist': u'Dying Fetus',
 u'similars': [[u'TRKGDUE128F92F0E83', 1],
  [u'TRKDLCR128F92F0E7F', 0.930181],
  [u'TRKMKOE128F93094A0', 0.170124],
  [u'TRYDFZL128F930949E', 0.162543],
  [u'TRKWQKG128F934BE03', 0.14108],
  [u'TRSWCRD128F428CDC0', 0.138215],
  [u'TRRQOBX12903CF5E9A', 0.136527],
  [u'TRGKQCB128F428CDC8', 0.132306],
  [u'TRIOJWC128F930D724', 0.131877],
  [u'TRYKETZ128F92C4061', 0.131466],
  [u'TROPLWL128F934BE0C', 0.129271],
  [u'TRKUOPO128F930D721', 0.122794],
  [u'TRLVJBD128F4290F80', 0.118457],
  [u'TRMRIIF128F92FE5ED', 0.101842],
  [u'TRGBOFX128F42774D4', 0.101842],
  [u'TRWWTTU128F92F1A0A', 0.100679],
  [u'TRXFRTR12903CF45B0', 0.0987389],
  [u'TRJYRBC128F934BDBF', 0.0977178],
  [u'TREDAJV12903CDD706', 0.0931674],
  [u'TRVHMHB128F4275500', 0.0918334],
  [u'TRTAVJM12903CF1DD1', 0.0916868],
  [u'TRPUBDK12903CF1DC9', 0.0907987],
  [u'TRPTWZH128F4275508', 0.0885968],
  [u'TRCJBLV12903D03E07', 0.0871354],
  [u'TRNOMEW12903D03E16', 0.0849296],
  [u'TRJERZV12903CF1E52', 0.0843221],
  [u'TRIISZE128F92FE5E1', 0.0819743],
  [u'TRPRCRM128F4290F82', 0.0813888],
  [u'TRSIKCJ12903CF4242', 0.0761096],
  [u'TRQGYBW128F93064AE', 0.0753326],
  [u'TRPKWLL12903CFA207', 0.0744523],
  [u'TRVZUTI128F428CE41', 0.0724359],
  [u'TRFRUJD128F427C208', 0.0714274],
  [u'TRKMEGQ128F4284795', 0.0706988],
  [u'TRTARAS12903CBE6D4', 0.0689639],
  [u'TRCMWCJ12903CDD85F', 0.0689617],
  [u'TRJZKWM12903CFA206', 0.0689617],
  [u'TRLAKKM128F423DACE', 0.0685598],
  [u'TRHNEMQ12903CBE6DE', 0.0666846],
  [u'TRXAGOI128F93064B8', 0.0660369],
  [u'TRNNIAW12903D01602', 0.0572348],
  [u'TRDHHIB128F4242267', 0.0503959],
  [u'TRPHMAG128F423DBFB', 0.0497767],
  [u'TRBOSUM12903CF8CCF', 0.043149],
  [u'TRNMWAE12903CF5CAB', 0.0383058],
  [u'TRMLBFT12903CF26AB', 0.0365885],
  [u'TRFZFSL128F934C3BA', 0.00421432],
  [u'TRXEPSX128F429103E', 0.00385149],
  [u'TRGSVKS128F4242183', 0.00370507],
  [u'TRXZKWA128F429103B', 0.00340451],
  [u'TRQCDZN128F934127B', 0.0030235],
  [u'TRLHKOV128F9341283', 0.0029796],
  [u'TRYNIAF128F934C683', 0.0029653],
  [u'TRUXEIW128F934C600', 0.00213755],
  [u'TRQUYRH12903CE7466', 0.00207372],
  [u'TRZYUKY12903CF5BC6', 0.00204379],
  [u'TRYGCEK128F934B511', 0.00203792],
  [u'TRWZDOO128F932D43A', 0.0020288],
  [u'TRLFSCJ128F92DC387', 0.00197012],
  [u'TRFQOJK12903CFF684', 0.00190903],
  [u'TRXCANO12903CA0AC9', 0.00184685],
  [u'TREKHBS128F934C5FA', 0.00182417],
  [u'TRETJRL12903CA0ACA', 0.00172702],
  [u'TRPZHYT128F93253C4', 0.00167472],
  [u'TREIPRB128F92DC38B', 0.00165596],
  [u'TRSVTFV128F93429F5', 0.00159807],
  [u'TRRQJCA128F934299C', 0.00159798],
  [u'TRFQWXP128F93429A5', 0.00159798],
  [u'TRYMYRX128F424EACF', 0.00159141],
  [u'TRXVADC12903CFF686', 0.00157535],
  [u'TRJXOKM128F9342A0D', 0.00156752],
  [u'TRUKHNI128F426FFF9', 0.00156286],
  [u'TRMWWSL128F92E6D25', 0.00150441],
  [u'TRSYCDY12903CF5CA5', 0.00149908],
  [u'TRTFWQU128F932D439', 0.00149077],
  [u'TRMAPOB128F427DAFD', 0.00139572],
  [u'TREUBPH128F427B26C', 0.00136202],
  [u'TRRWKJM128F934B5DE', 0.00131633],
  [u'TRDCFGN128F934B5DA', 0.00131632],
  [u'TRYZAGJ128F424EAD2', 0.00131602],
  [u'TRIZEGF12903D09FCF', 0.0013008],
  [u'TRLEERZ128F426B204', 0.00126853]],
 u'tags': [[u'Technical Death Metal', u'100'],
  [u'Brutal Death Metal', u'85'],
  [u'death metal', u'42'],
  [u'grindcore', u'28'],
  [u'metal', u'14'],
  [u'death', u'14'],
  [u'Technical Brutal Death Metal', u'14'],
  [u'Ethos Of Coercion', u'14'],
  [u'nosz kurwa jakie stopy', u'14']],
 u'timestamp': u'2011-07-30 16:58:31.240734',
 u'title': u'Ethos of Coercion',
 u'track_id': u'TRMMMQY128F92F0EA3'}